# Scheduling: P | p - batch, incompatible | TWT
In diesem Notebook soll das obige Ablaufplanungsproblem mit einer Anwendung der VNS-Metaheuristik (Variable Neighborhood Search) gelöst werden. Das Problem ist eine Verallgemeinerung vom Problem 1 || TWT, welches NP-hart ist. Eine Metaheuristik kann folglich zur effizienten Lösung des Problems angewendet werden.

Zuerst generieren wir ein zufälliges Setup von parallen Maschinen, Produktfamilien mit eigenen Laufzeiten und Jobs.

In [1110]:
from typing import List, Dict, Tuple
import random
from scheduling_lib import Job, Batch, BatchMachine

BATCH_SIZE = 10
MIN_FAMILIES = 10
MAX_FAMILIES = 20
MIN_PROCESSING_TIME = 2
MAX_PROCESSING_TIME = 30
MAX_WEIGHT = 10
MIN_JOBS = 50
MAX_JOBS = 500
MIN_DUE_DATE = 4
MAX_DUE_DATE = 100
MIN_MACHINES = 2
MAX_MACHINES = MIN_FAMILIES

# Generate random data
# list of family processing times
p_f = [random.randint(MIN_PROCESSING_TIME, MAX_PROCESSING_TIME) for i in range(random.randint(MIN_FAMILIES, MAX_FAMILIES))]
print("Family processing times:\n" + str(p_f))

# list of machines
machines = [BatchMachine() for i in range(random.randint(MIN_MACHINES, MAX_MACHINES))]
print("Number of machines:\n" + str(len(machines)))

# list of jobs
jobs = [Job(family=random.randint(0, len(p_f) - 1), weight=random.randint(1, MAX_WEIGHT), due_date=random.randint(MIN_DUE_DATE, MAX_DUE_DATE)) for i in range(random.randint(MIN_JOBS, MAX_JOBS))]
print("Number of jobs:\n" + str(len(jobs)))

Family processing times:
[3, 20, 6, 30, 6, 9, 16, 17, 3, 26, 10, 24, 20, 2, 24, 18, 29, 6]
Number of machines:
4
Number of jobs:
438


TWT Funktion: $\sum_{j=1}^n w_j T_j = \sum_{j=1}^n w_j \max\{C_j - d_j; 0\}$

In [1111]:
def twt(jobs: list[Job]) -> int:
    return sum(job.w * max(job.C - job.d, 0) for job in jobs)

In [1112]:
from math import exp


def atc(job: Job, p_avg: float, t: int, kappa: int) -> int:
    return (job.w / p_f[job.f]) * kappa * exp(-(max(job.d - p_f[job.f] - t, 0))/(kappa * p_avg))

## Eröffnungsheuristik: ATC-BATC-Heuristic

In [1113]:
from statistics import mean
import copy
from pprint import pprint

initial_schedule = list()
top_twt = None
top_kappa = None

# iterate over different values of kappa
for kappa in [0.5 * l for l in range(1, 11)]:
    # get a copy of the machines, jobs and schedule to not modify the original data while tuning kappa
    temp_machines = machines.copy()
    temp_jobs = jobs.copy()
    temp_schedule = list()
    # start time
    t = 0
    # iterate over the jobs
    while len(temp_jobs) > 0:
        # remove all batches that are finished from the machines
        for machine in temp_machines: machine.update(t)
        # check if there are machines that are not allocated to a batch
        allocatable_machines = [machine for machine in temp_machines if machine.is_idle()]
        # iterate over the free machines
        while len(allocatable_machines) > 0 and len(temp_jobs) > 0:
            # sort all remaining jobs by their ATC value descending
            p_avg = mean([p_f[job.f] for job in temp_jobs])
            for job in temp_jobs:
                job.set_metrics({"atc": atc(job, p_avg, t, kappa)})
            temp_jobs.sort(
                key = lambda job: job.get_metrics()["atc"],
                reverse = True
            )
            # get jobs of families that are currently not in a batch
            allocated_families = [machine.batch.f for machine in temp_machines if not machine.is_idle()]
            allocatable_families = [i for i in range(len(p_f)) if i not in allocated_families]
            # create batches per family by picking the best jobs by ATC Index
            allocatable_batches = list()
            for family in allocatable_families:
                family_batch = Batch()
                family_batch.f = family
                family_batch.p = p_f[family]
                # get all remaining jobs of the family
                family_jobs = [job for job in temp_jobs if job.f == family]
                # select min(remaining jobs in family, BATCH_SIZE) jobs of the family
                family_batch.jobs = family_jobs[:min(len(family_jobs), BATCH_SIZE)]
                family_batch.set_metrics({"batc": sum([job.get_metrics()["atc"] for job in family_batch.jobs])})
                allocatable_batches.append(family_batch)
            # skip if there are no batches to allocate
            if len(allocatable_batches) == 0:
                continue
            # sort the batches by their BATC value descending
            allocatable_batches.sort(
                key = lambda batch: batch.get_metrics()["batc"],
                reverse = True
            )
            # chose the best batch, add its completion time and allocate it to the first available machine
            top_batch = allocatable_batches[0]
            top_batch.C = t + p_f[top_batch.f]
            top_batch.apply_C_to_jobs()
            allocatable_machines[0].batch
            # remove the jobs of the batch from the remaining jobs
            for job in top_batch.jobs:
                temp_jobs.remove(job)
            # add the batch to the schedule
            temp_schedule.append({"batch": top_batch, "machine": temp_machines.index(allocatable_machines[0]), "start": t, "end": top_batch.C})
            # remove the machine from the list of allocatable machines
            allocatable_machines.remove(allocatable_machines[0])
        # increment time
        t += 1
    # get the TWT of the schedule
    temp_twt = twt([job for batch in temp_schedule for job in batch["batch"].jobs])
    if top_twt is None or temp_twt < top_twt:
        initial_schedule = temp_schedule
        top_twt = temp_twt
        top_kappa = kappa
    print("current kappa: " + str(kappa) + ", current TWT: " + str(temp_twt) + ", top kappa: " + str(top_kappa) + ", top TWT: " + str(top_twt))

# add batches to machine queues according to the initial schedule
machine_queues = list()
for i in range(len(machines)):
    machine_queues.append(
        [element["batch"] for element in initial_schedule if element["machine"] == i]
    )

print(f"\nBatch composition with kappa={top_kappa} and twt={top_twt}")
for s in [f"Machine {idx:>2}: {str(len(q)):>3} Batches, done at {q[-1].C:>7}" for idx, q in enumerate(machine_queues)] : print(s)


current kappa: 0.5, current TWT: 3255, top kappa: 0.5, top TWT: 3255
current kappa: 1.0, current TWT: 3435, top kappa: 0.5, top TWT: 3255
current kappa: 1.5, current TWT: 3674, top kappa: 0.5, top TWT: 3255
current kappa: 2.0, current TWT: 3881, top kappa: 0.5, top TWT: 3255
current kappa: 2.5, current TWT: 3987, top kappa: 0.5, top TWT: 3255
current kappa: 3.0, current TWT: 4098, top kappa: 0.5, top TWT: 3255
current kappa: 3.5, current TWT: 4234, top kappa: 0.5, top TWT: 3255
current kappa: 4.0, current TWT: 4300, top kappa: 0.5, top TWT: 3255
current kappa: 4.5, current TWT: 4352, top kappa: 0.5, top TWT: 3255
current kappa: 5.0, current TWT: 4411, top kappa: 0.5, top TWT: 3255

Batch composition with kappa=0.5 and twt=3255
Machine  0:  13 Batches, done at      22
Machine  1:  13 Batches, done at      36
Machine  2:  13 Batches, done at      29
Machine  3:  12 Batches, done at      35


## VNS-Verfahren

In [1114]:
# helper function for choosing a random machine queue
def random_machine_queues() -> Tuple[List[Batch], List[Batch]]:
    a = random.choice(machine_queues)
    b = random.choice([q for q in machine_queues if q != a])
    return a, b

# helper function for choosing a random machine queue and an index 
def random_machine_queues_and_indices() -> Tuple[Tuple[List[Batch], int], Tuple[List[Batch], int]]:
    a, b = random_machine_queues()
    index_a, index_b = random.randint(0, len(a) - 1), random.randint(0, len(b) - 1)
    return (a, index_a), (b, index_b)

# helper function for updating the completion times of batches
def update_completion_times(queue: List[Batch], index: int):
    for i in range(index, len(queue)):
        # add the processing time of the batch to the completion time of the previous batch
        queue[i].C = queue[i].p
        if i > 0:
            queue[i].C += queue[i - 1].C
        # apply the completion time to the jobs of the batch
        queue[i].apply_C_to_jobs()

# Neighborhood 1: Swap two batches
def swap_batch(r: int):
    for i in range(r):
        # get two random machine queues and their indices
        (a, index_a), (b, index_b) = random_machine_queues_and_indices()
        # swap the batches at the indices
        a[index_a], b[index_b] = b[index_b], a[index_a]
        # update the completion times of the batches
        update_completion_times(a, index_a)
        update_completion_times(b, index_b)

# Neighborhood 2: Swap two batch sequences
def swap_seq(r: int):
    # get two random machine queues and their indices
    (a, index_a), (b, index_b) = random_machine_queues_and_indices()
    # swap the batch sequences of length r at the indices
    a[index_a:index_a+r], b[index_b:index_b+r] = b[index_b:index_b+r], a[index_a:index_a+r]
    # update the completion times of the batches
    update_completion_times(a, index_a)
    update_completion_times(b, index_b)

# Neighborhood 3: Move a batch
def move_batch(r):
    for i in range(r):
        # get two random machine queues and their indices
        (a, index_a), (b, index_b) = random_machine_queues_and_indices()
        # remove the batch from the first queue and insert it into the second queue
        b.insert(index_b, a.pop(index_a))
        # update the completion times of the batches
        update_completion_times(a, index_a)
        update_completion_times(b, index_b)

# Neighborhood 4: Move a batch sequence
def move_seq(r):
    # get two random machine queues and their indices
    (a, index_a), (b, index_b) = random_machine_queues_and_indices()
    # remove the batch sequence of length r from the first queue and insert it into the second queue
    for i in range(r):
        if len(a) <= index_a:
            break
        b.insert(index_b+r, a.pop(index_a))
    # update the completion times of the batches
    update_completion_times(a, index_a)
    update_completion_times(b, index_b)

In [1124]:
import itertools


def balance_workload(machine_queues: List[List[Batch]] = machine_queues):
    while True:
        # get max C of all queues
        C_q_max = [queue[-1].C for queue in machine_queues]
        # get the index r of the queue with max C
        r = C_q_max.index(max(C_q_max))
        # get the index r of the queue with min C
        s = C_q_max.index(min(C_q_max))
        # save total C max
        C_max = C_q_max[r]
        # move last batch from queue r to queue s if C max decreases
        temp_machine_queues = copy.deepcopy(machine_queues)
        temp_machine_queues[s].append(temp_machine_queues[r].pop())
        update_completion_times(temp_machine_queues[r], len(temp_machine_queues[r]) - 1)
        update_completion_times(temp_machine_queues[s], len(temp_machine_queues[s]) - 1)
        if max([queue[-1].C for queue in temp_machine_queues]) < C_max:
            print("here")
            machine_queues = temp_machine_queues
        else:
            break

def decomposition_heuristic(ld: int, alpha: int, iter_max: int, machine_queue: List[Batch] = machine_queues):
    b_min = 0
    # iterate until iter_max iterations
    for i in range(iter_max):
        # select first ld batches to apply full twt enumeration to
        phi = machine_queue[b_min:min(ld, len(machine_queue))]
        # preset phi_optimal to phi
        phi_optimal = copy.deepcopy(phi)
        if(len(phi) > 0):
            # apply full twt enumeration to the first ld batches
            top_twt = twt([job for batch in phi for job in batch.jobs])
            for p in itertools.permutations(phi):
                permutation = list(p)
                for batch in permutation:
                    # update the completion times of the batches as if t were 0
                    update_completion_times(permutation, 0)
                # calculate the twt of the permutation
                temp_twt = twt([job for batch in permutation for job in batch.jobs])
                # if the twt is smaller than the current best twt, update the best twt and the best permutation
                if top_twt is None or temp_twt < top_twt:
                    top_twt = temp_twt
                    # take over first alpha batches from best permutation
                    phi_optimal = copy.deepcopy(permutation)
            # add the best permutation to the machine queue
            machine_queue[b_min:min(ld, len(machine_queue))] = phi_optimal
            # increment b_min to effectively add the first alpha batches to the machine queue
            b_min += min(alpha, len(phi))
            update_completion_times(phi, 0)
        else:
            break
# TODO     
def swap(machine_queues: List[List[Batch]] = machine_queues):
    S_f = set()
    for index, p in enumerate(p_f):
        B_f_without_S_f = set([batch for queue in machine_queues for batch in queue]) - S_f
        b = min([batch for batch in B_f_without_S_f])
        for job in [batch.jobs for batch in B_f_without_S_f]
        
        
swap()

{Batch(f=10, p=10, jobs=[Job(f=10, w=8, d=13, C=12, metrics={'atc': 3.9558995747820056}), Job(f=10, w=7, d=14, C=12, metrics={'atc': 3.4232496912600996}), Job(f=10, w=10, d=20, C=12, metrics={'atc': 4.575643051783082}), Job(f=10, w=6, d=17, C=12, metrics={'atc': 2.8382300116093044}), Job(f=10, w=6, d=17, C=12, metrics={'atc': 2.8382300116093044}), Job(f=10, w=8, d=21, C=12, metrics={'atc': 3.620156880580587}), Job(f=10, w=5, d=24, C=12, metrics={'atc': 2.1885839426207676}), Job(f=10, w=4, d=30, C=14, metrics={'atc': 1.7099137732785916}), Job(f=10, w=2, d=26, C=14, metrics={'atc': 0.8891154585871319}), Job(f=10, w=9, d=39, C=12, metrics={'atc': 3.335912982065085})], C=11, metrics={'batc': 1.2134393354333564}), Batch(f=17, p=6, jobs=[Job(f=17, w=7, d=7, C=8, metrics={'atc': 5.833333333333334}), Job(f=17, w=3, d=12, C=8, metrics={'atc': 2.389545010116531}), Job(f=17, w=4, d=15, C=8, metrics={'atc': 3.0798910735925173}), Job(f=17, w=5, d=23, C=8, metrics={'atc': 3.517189626578463}), Job(f=